If in VSCODE:

In [1]:
# check if Torch supports CUDA
import torch
print(torch.__version__)
torch.cuda.is_available()


2.1.0+cu121


True

Hyper params

In [1]:
# The model that you want to train from the Hugging Face hub
model_name = "codellama/CodeLlama-7b-Instruct-hf"

# The instruction dataset to use
dataset_name = "StarkWizard/cairo-instruct"

# Fine-tuned model name
new_model = "StarkWizard/llama-2-7b-cairo-trained-PEFT"


import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"



nb_epochs = 3

In [2]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="codellama-cairo",
    
    # track hyperparameters and run metadata
    config={

    "epochs":nb_epochs,
    }
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pierre-emmanuel-chaut. Use `wandb login --relogin` to force relogin


Load Base Model and Tokenizer

In [3]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM,prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTTrainer
import os

# Load the tokenizer from the model (llama2)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True, add_eos_token=True, use_fast=False)
tokenizer.padding_side = "right"
tokenizer.pad_token_id = 18610

# load the quantized settings: 4 bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    quantization_config=bnb_config,
    max_memory= {i: '12000MB' for i in range(torch.cuda.device_count())},
    device_map={"": 0}
)



# don't use the cache
model.config.use_cache = False
model.config.pretraining_tp=1
model.config.window = 512 
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
        r=64, lora_alpha=16, lora_dropout=0.1, bias="none", task_type="CAUSAL_LM", target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "up_proj",
        "down_proj",
        "gate_proj",
        'lm_head',],

        inference_mode = False
    )



/home/pechaut/miniconda3/envs/cairo-llm/lib/python3.9/site-packages/transformers/models/auto/tokenization_auto.py:655: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Import Dataset from Hugging Face

In [4]:
from datasets import load_dataset

# Load the dataset
dataset_train = load_dataset(dataset_name, split="train", download_mode='force_redownload',ignore_verifications=True)
dataset_test = load_dataset(dataset_name, split="eval", download_mode='force_redownload',ignore_verifications=True)
dataset_train = dataset_train.shuffle(seed=42)

/home/pechaut/miniconda3/envs/cairo-llm/lib/python3.9/site-packages/datasets/load.py:2097: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3226 [00:00<?, ? examples/s]

Generating eval split:   0%|          | 0/5 [00:00<?, ? examples/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3226 [00:00<?, ? examples/s]

Generating eval split:   0%|          | 0/5 [00:00<?, ? examples/s]

Check to see if everything is fine before launching training

Load Model

In [6]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
from trl import SFTTrainer
import transformers



supervised_finetuning_trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    packing=False,
    max_seq_length=1024,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
         gradient_checkpointing = True,
         evaluation_strategy="steps",
         max_grad_norm=0.3,
        warmup_ratio=0.03,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="constant",
        save_strategy="epoch",
        logging_steps=50,
        num_train_epochs=nb_epochs,
        output_dir=new_model,
        fp16=False,
        push_to_hub=True,
        group_by_length=True,
         report_to="wandb",
         adam_beta2=0.999,
         do_train=True,
    ),
)


Map:   0%|          | 0/3226 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [6]:
torch.cuda.empty_cache()
supervised_finetuning_trainer.train()
supervised_finetuning_trainer.model.push_to_hub(new_model)
tokenizer.push_to_hub(new_model)

  0%|          | 0/4839 [00:00<?, ?it/s]

/home/pechaut/miniconda3/envs/cairo-llm/lib/python3.9/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 1.7399, 'learning_rate': 0.0002, 'epoch': 0.03}


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 1.9637229442596436, 'eval_runtime': 0.9674, 'eval_samples_per_second': 5.168, 'eval_steps_per_second': 1.034, 'epoch': 0.03}
{'loss': 1.4441, 'learning_rate': 0.0002, 'epoch': 0.06}


...

{'eval_loss': 1.2225720882415771, 'eval_runtime': 1.043, 'eval_samples_per_second': 4.794, 'eval_steps_per_second': 0.959, 'epoch': 2.94}
{'loss': 0.5001, 'learning_rate': 0.0002, 'epoch': 2.98}


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 1.2060960531234741, 'eval_runtime': 1.023, 'eval_samples_per_second': 4.888, 'eval_steps_per_second': 0.978, 'epoch': 2.98}
{'train_runtime': 11601.8277, 'train_samples_per_second': 0.834, 'train_steps_per_second': 0.417, 'train_loss': 0.7996881070721369, 'epoch': 3.0}


adapter_model.bin:   0%|          | 0.00/649M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/StarkWizard/llama-2-7b-cairo-trained-PEFT/commit/d2bcffe7f63ed299b7920939d1056447480fa7b4', commit_message='Upload tokenizer', commit_description='', oid='d2bcffe7f63ed299b7920939d1056447480fa7b4', pr_url=None, pr_revision=None, pr_num=None)

---
Merge here or use the dedicated notbook

---

In [7]:
from transformers import AutoModelForCausalLM
from peft import PeftModel
import torch
from transformers import  AutoTokenizer, BitsAndBytesConfig

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_name,
                                             trust_remote_code=True,
                                             low_cpu_mem_usage=True,
                                             device_map={"": "cpu"},
                                             torch_dtype=torch.float16
                                             )

model_to_merge  = PeftModel.from_pretrained(model, new_model,
                        torch_dtype=torch.float16, 
                        device_map={"": "cpu"}
                         )
merged_model = model_to_merge.merge_and_unload()
#model.save_pretrained("cairo-mistral")
merged_model.push_to_hub(hub_name,max_shard_size="1GB")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

NameError: name 'hub_name' is not defined